In [1]:
import os
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Parámetros de filtrado
UMBRAL_BASE = 0.60  
UMBRAL_MAXIMO = 0.45  
MIN_DOCUMENTOS_RELEVANTES = 1  

def cargar_embeddings_desde_archivo(ruta_archivo):
    """Carga los embeddings desde un archivo JSON."""
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data["archivo"], data["secciones"]

def calcular_similitud(embedding1, embedding2):
    """Calcula la similitud mediante la distancia del coseno."""
    return cosine(embedding1, embedding2)

def buscar_documentos_similares(texto, carpeta_embeddings):
    """
    Busca documentos en una carpeta que sean más similares al texto de entrada.
    Retorna una lista de tuplas con el nombre del archivo, la menor distancia y el párrafo más similar.
    """
    embedding_texto = embedding_model.encode(texto)
    resultados = {}

    for archivo in os.listdir(carpeta_embeddings):
        if archivo.endswith("_embeddings.json"):
            ruta_archivo = os.path.join(carpeta_embeddings, archivo)
            nombre_archivo, secciones = cargar_embeddings_desde_archivo(ruta_archivo)

            menor_distancia = float("inf")
            parrafo_mas_similar = ""

            for seccion in secciones:
                embedding_parrafo = np.array(seccion["embedding"])
                distancia = calcular_similitud(embedding_texto, embedding_parrafo)

                if distancia < menor_distancia:
                    menor_distancia = distancia
                    parrafo_mas_similar = seccion["texto"]

            resultados[nombre_archivo] = (menor_distancia, parrafo_mas_similar)

    resultados_ordenados = sorted(resultados.items(), key=lambda x: x[1][0])

    umbral_actual = UMBRAL_BASE
    documentos_relevantes = [(archivo, distancia, parrafo) for archivo, (distancia, parrafo) in resultados_ordenados if distancia <= umbral_actual]

    if len(documentos_relevantes) < MIN_DOCUMENTOS_RELEVANTES:
        umbral_actual = UMBRAL_MAXIMO
        documentos_relevantes = [(archivo, distancia, parrafo) for archivo, (distancia, parrafo) in resultados_ordenados if distancia <= umbral_actual]

    if len(documentos_relevantes) < MIN_DOCUMENTOS_RELEVANTES:
        return []

    return documentos_relevantes

def construir_respuesta(texto_pregunta, carpeta_embeddings):
    """
    Construye una respuesta en lenguaje natural combinando los párrafos más similares.
    """
    documentos_relevantes = buscar_documentos_similares(texto_pregunta, carpeta_embeddings)

    if not documentos_relevantes:
        return "No se encontró una respuesta clara en los documentos."

    respuesta = "Con base en los documentos analizados, la respuesta es:\n\n"

    # Unir los mejores párrafos asegurando coherencia
    for _, _, parrafo in documentos_relevantes[:3]:  # Tomar hasta 3 párrafos relevantes
        respuesta += f"{parrafo}\n\n"

    return respuesta.strip()

# Ejemplo de uso
carpeta_embeddings = "data"
texto_pregunta = "All the information about Burkina?"

respuesta = construir_respuesta(texto_pregunta, carpeta_embeddings)

# Mostrar la respuesta generada
print("\n🔹 Respuesta generada:")
print(respuesta)


c:\Users\arodri\Proyectos\TFM_2025\TFM2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔹 Respuesta generada:
Con base en los documentos analizados, la respuesta es:

Efficiency and Feasibility: Best Use of Resources
With the Burkina Faso HRP being severely underfunded, it is unclear whether this anticipatory
action pilot diverts funding away from more urgent needs identified by the Food Security Cluster.
● Recommendation: Discuss the potential impacts that $15 million could have on the
region or globally and compare to the impacts of the no-regrets approach.
